# Regression Classification - Traffic Collisions


See if weather conditions can help us predict the number of collisions in Los Angeles (we already know that rain, etc. can cause collisions, but we will see heat/humidity can make drivers impatient and cause collisions). Regression classification task: predict the number of collisions observed in a given day (i.e., the `collision` column in the data set) based on that weather readings of that day. This is important, because this model can enable law enforcement to be proactive in handling collisions.

## Goal

Use the **collision.csv** data set and build a model to predict **collision**. 

# Read and Prepare the Data

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

collision = pd.read_csv("collision.csv")

#drop date column
collision = collision.drop(['Date'], axis=1)
collision.head()

,Ave humudity,Ave pressure,Ave temp,Ave wind speed,day of week,collision
0,36.875000,1027.291667,291.007083,0.791667,Sunday,98
1,70.833333,1011.250000,298.941667,2.083333,Tuesday,153
2,71.583333,1015.791667,291.423723,1.875000,Monday,110
3,69.208333,1011.833333,289.163987,1.541667,Sunday,158
4,70.458333,1012.666667,295.560000,1.875000,Saturday,172


In [3]:
#Check for missing values
collision.isna().sum()

Ave humudity      0
Ave pressure      0
Ave temp          0
Ave wind speed    0
day of week       0
collision         0
dtype: int64

### Split into Train/Test

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(collision, test_size=0.3)

### Separate the target variable

In [5]:
train_y = train[['collision']]
test_y = test[['collision']]

train_inputs = train.drop(['collision'], axis=1)
test_inputs = test.drop(['collision'], axis=1)

### Identify Numeric/Categorical Columns

In [7]:
#Identify data types
train_inputs.dtypes

Ave humudity      float64
Ave pressure      float64
Ave temp          float64
Ave wind speed    float64
day of week        object
dtype: object

In [8]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns

numeric_columns

Index(['Ave humudity', 'Ave pressure', 'Ave temp', 'Ave wind speed'], dtype='object')

In [9]:
# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns

categorical_columns

Index(['day of week'], dtype='object')

### Pipeline

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

numeric_transformer = Pipeline(steps=[
                ('scaler', StandardScaler())])

In [12]:
categorical_transformer = Pipeline(steps=[   
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [13]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)])

### Transform Train/Test

In [15]:
#Fit transform the train data
train_x = preprocessor.fit_transform(train_inputs)
train_x

array([[ 0.57653537,  1.49693035, -0.85664084, ...,  0.        ,
         0.        ,  0.        ],
       [-0.23152034, -0.20691301,  0.24764842, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.37202744, -0.47404106,  0.27519402, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.16253152, -4.51705932, -1.30148917, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.81346529,  0.10714287,  1.26874659, ...,  0.        ,
         0.        ,  0.        ],
       [-0.51583624, -0.35130655,  1.61183988, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
train_x.shape

(897, 11)

In [17]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)
test_x

array([[-1.06950405, -0.25384093,  1.07617448, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.01298522,  1.32365814, -2.11984912, ...,  0.        ,
         0.        ,  0.        ],
       [-0.66547619, -0.41267376,  0.38713622, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.27725548,  0.08909363, -0.92802387, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.42190743, -0.21413275, -0.78224393, ...,  0.        ,
         0.        ,  1.        ],
       [-1.90748775,  1.31282862,  0.41490353, ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
test_x.shape

(385, 11)

### Determine Baseline

In [20]:
#Find the average collisions
mean_value = np.mean(train_y['collision'])
mean_value

140.53957636566332

In [22]:
from sklearn.metrics import mean_squared_error

# Predict all values as the mean
baseline_pred = np.repeat(mean_value, len(test_y))

#Calculate baseline RMSE
baseline_mse = mean_squared_error(test_y, baseline_pred)
baseline_rmse = np.sqrt(baseline_mse)
print('Baseline RMSE: {}' .format(baseline_rmse))

Baseline RMSE: 29.39794772936068


***Baseline prediction has an error of 29.4 collisions***

# Model 1 - ElasticNet SGD


In [39]:
from sklearn.linear_model import SGDRegressor 
from sklearn.linear_model import ElasticNet

elastic = SGDRegressor(max_iter=200, penalty='elasticnet', l1_ratio=0.9, alpha = 0.1, 
                          eta0=0.01, tol=0.0001)
elastic.fit(train_x, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SGDRegressor(alpha=0.1, l1_ratio=0.9, max_iter=200, penalty='elasticnet',
             tol=0.0001)

In [40]:
#Train RMSE
train_pred = elastic.predict(train_x)

train_mse = mean_squared_error(train_y, train_pred)

train_rmse = np.sqrt(mean_squared_error (train_y, train_pred))

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 26.473849446367602


In [41]:
#Test RMSE
test_pred = elastic.predict(test_x)

test_mse = mean_squared_error (test_y, test_pred)

test_rmse = np.sqrt(mean_squared_error (test_y, test_pred))

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 27.24108483675306


# Model 2 - SVM (rbf)

In [131]:
from sklearn.svm import SVR 
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import random

#define ranges for C and gamma
param_distribs = {
        'C': randint(low=1, high=100),
        'gamma': uniform(0.01, 0.5),   
        'epsilon': uniform(0.01, 0.1)
}

#define model
rbf_svm = SVR(kernel="rbf")

#search variations of hyperparameters 50 iterations
rbf_search = RandomizedSearchCV(rbf_svm, param_distributions=param_distribs,
                                n_iter=1000, cv=10, scoring='neg_root_mean_squared_error', random_state=42, verbose=True)

#fit models on training data
rbf_search.fit(train_x, train_y)

#svm = SVR(kernel="rbf", C=10, epsilon=0.01, gamma='scale') 

#svm.fit(train_x, train_y)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

RandomizedSearchCV(cv=10, estimator=SVR(), n_iter=1000,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D11843D790>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D11CC1DD00>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D11843D610>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=True)

In [133]:
#Determine best model params
rbf_model = rbf_search.best_estimator_
rbf_search.best_params_

{'C': 92, 'epsilon': 0.057667682833036846, 'gamma': 0.03218833176661839}

In [134]:
#Train RMSE
train_pred = rbf_model.predict(train_x)

train_mse = mean_squared_error(train_y, train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 25.040761791687157


In [135]:
#Test RMSE
test_pred = rbf_model.predict(test_x)

test_mse = mean_squared_error(test_y, test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 26.722983368300003


# Model 3 - LinearSVR

In [58]:
from sklearn.svm import LinearSVR 

linSVR = LinearSVR(C=10, epsilon=0.5, max_iter=10000) 

linSVR.fit(train_x, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LinearSVR(C=10, epsilon=0.5, max_iter=10000)

In [59]:
#Train RMSE
train_pred = linSVR.predict(train_x)

train_mse = mean_squared_error(train_y, train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 26.58843961043037


In [60]:
#Test RMSE
test_pred = linSVR.predict(test_x)

test_mse = mean_squared_error(test_y, test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 27.276817929470322


# Model 4 - MLPRegressor

In [101]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {
'learning_rate': ["constant","invscaling", "adaptive"],
'hidden_layer_sizes': [(75,50,25,10),(70,60,50,40),(60,50,40,30),(50,40,30)],
'alpha': [.001,.01,.1],
'activation': ["identity","relu","tanh"],
'solver': ['lbfgs','adam']
}

#2-fold cross validation
mlp = GridSearchCV(estimator=MLPRegressor(),
                      param_grid=param_grid,
                      n_jobs=-1,
                      cv=2, 
                      scoring='neg_root_mean_squared_error',
                      return_train_score=True,
                      verbose=True)

mlp.fit(train_x, train_y)

Fitting 2 folds for each of 216 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.5min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=2, estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['identity', 'relu', 'tanh'],
                         'alpha': [0.001, 0.01, 0.1],
                         'hidden_layer_sizes': [(75, 50, 25, 10),
                                                (70, 60, 50, 40),
                                                (60, 50, 40, 30),
                                                (50, 40, 30)],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive'],
                         'solver': ['lbfgs', 'adam']},
             return_train_score=True, scoring='neg_root_mean_squared_error',
             verbose=True)

In [102]:
#Determine best model params
mlp_model = mlp.best_estimator_
mlp.best_params_

{'activation': 'relu',
 'alpha': 0.01,
 'hidden_layer_sizes': (70, 60, 50, 40),
 'learning_rate': 'adaptive',
 'solver': 'adam'}

In [103]:
#Train RMSE
train_pred = mlp_model.predict(train_x)

train_mse = mean_squared_error(train_y, train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 24.117751873425263


In [104]:
#Test RMSE
test_pred = mlp_model.predict(test_x)

test_mse = mean_squared_error(test_y, test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 27.35780293037864


# Model 5 - Decision Tree

In [127]:
from sklearn.tree import DecisionTreeRegressor

param_grid = [
    {'min_samples_leaf': np.arange(3, 30), 
     'max_depth': np.arange(3,30)
    }
]

tree_reg = DecisionTreeRegressor()

dt = GridSearchCV(tree_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=1,
                           return_train_score=True)

dt.fit(train_x, train_y)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3645 out of 3645 | elapsed:   19.5s finished


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
                          'min_samples_leaf': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29])}],
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [128]:
#Determine best model params
dt_model = dt.best_estimator_
dt.best_params_

{'max_depth': 4, 'min_samples_leaf': 11}

In [129]:
#Train RMSE
train_pred = dt_model.predict(train_x)

train_mse = mean_squared_error(train_y, train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 25.591083916234172


In [130]:
#Test RMSE
test_pred = dt_model.predict(test_x)

test_mse = mean_squared_error(test_y, test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 27.754967144264803


### Prepare Competition Dataset

In [140]:
comp = pd.read_csv('collision_competition.csv')
#drop date column
comp_set = comp.drop(['Date','collision'], axis=1)
comp_set.head()

,Ave humudity,Ave pressure,Ave temp,Ave wind speed,day of week
0,55.208333,1022.208333,297.793292,0.625000,Friday
1,57.416667,1017.125000,286.745373,2.250000,Thursday
2,35.041667,1030.333333,292.555500,0.791667,Sunday
3,37.208333,1012.791667,290.765625,0.916667,Wednesday
4,68.791667,1013.291667,294.860000,1.041667,Sunday


In [158]:
# Transform the comp data
comp_trans = preprocessor.transform(comp_set)
comp_trans.shape

(200, 11)

In [151]:
#Predict comp data set with best model
comp_pred = rbf_model.predict(comp_trans)

In [154]:
#Drop everything except date and collision from comp dataset
comp_final = comp[['Date','collision']]

In [155]:
#Add the predictions to the collision column to final dataframe
comp_final['collision'] = comp_pred
comp_final.head()

<ipython-input-155-16df0f1c4586>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_final['collision'] = comp_pred


,Date,collision
0,9/12/2014,155.308563
1,3/31/2016,150.565405
2,3/15/2015,128.342658
3,1/22/2014,137.051204
4,8/20/2017,125.054384


In [157]:
#Output to CSV
comp_final.to_csv("johnson_collision_competition.csv",index=False)